# Race Track

We use Build123d to create a 3D model of a race track. We export the 3D model as a stl file.

We also export a 2D projection as svg file. We use image magic to convert the svg file to a pgn file and then to convert the png file to a pgm file. Adding some meta data will provide us with a map that we can use in Nav2

In [1]:
from jupyter_cadquery import show, open_viewer, set_defaults
import cadquery as cq
from build123d import *
cv = open_viewer("Build123d", cad_width=770, glass=True)
set_defaults(edge_accuracy=0.0001)

Overwriting auto display for cadquery Workplane and Shape


In [2]:
class SideWall():
    
    def __init__(self, whichWall):
        ext_r = 150
        int_r = 50
        length = 1000
        width = 551

        len1 = 700
        arc1 = 180
        len2 = 150
        arc2 = -90
        len3 = 50
        arc3 = 360 - 168.2
        len3 = 50
        len4 = 58.78
        arc4 = -122.2
        len5 = 103.05
        arc5 = 360 - 159.6

        wall_hight = 20
        wall_thick = 2
        
        if whichWall == 'ext':
            r1 = ext_r
            r2 = int_r
        else:
            r1 = int_r
            r2 = ext_r

        with BuildPart() as self.p:
            with BuildLine(mode=Mode.PRIVATE) as l:
                base_line = Line((0,0),(len1,0))
                rigth_curve = JernArc(start=base_line @ 1, tangent=base_line % 1, radius=r1, arc_size=arc1)
                line2 = PolarLine(rigth_curve @ 1, len2, direction=rigth_curve % 1)
                rigth_upper_curve = JernArc(start=line2 @ 1, tangent=line2 % 1, radius=r2, arc_size=arc2)
                line3 = PolarLine(rigth_upper_curve @ 1, len3, direction=rigth_upper_curve%1)
                upper_curve = JernArc(start=line3 @ 1, tangent=line3 % 1, radius=r1, arc_size=arc3)
                line4 = PolarLine(upper_curve @ 1, len4, direction=upper_curve%1)
                rigth_upper_curve = JernArc(start=line4 @ 1, tangent=line4%1, radius=r2, arc_size=arc4)
                line5 = PolarLine(rigth_upper_curve @ 1, len5, direction=rigth_upper_curve%1)
                left_curve = JernArc(start=line5 @ 1, tangent=line5%1, radius=r1, arc_size=arc5)
            with BuildSketch(Plane.YZ) as s:
                Rectangle(wall_thick, wall_hight)
            sweep(path=l.wires()[0])

In [3]:
p_int = SideWall('int')
p_ext = SideWall('ext')

In [4]:
p_int.p.part.label = "internal wall"
p_int.p.part.location = Location((0, 100, 0))
p_ext.p.part.label = "external wall"

racetrack_assembly = Compound(label="racetrack", children=[p_int.p.part, p_ext.p.part])

In [5]:
racetrack_assembly

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.06s


In [6]:
racetrack_assembly.export_stl('racetrack.stl')

True

In [7]:
racetrack_assembly.export_svg('racetrack.svg', (0, 0, 1000000), (0, 1, 0), 
                          svg_opts={"pixel_scale": 1, "margin_left": 0, "margin_top": 0,"show_axes": False, "show_hidden": False})

In [8]:
import xml.etree.ElementTree as ET
tree = ET.parse('racetrack.svg')
root = tree.getroot()
dim = root.attrib

In [9]:
!convert racetrack.svg racetrack.png

In [10]:
!convert racetrack.png -flatten racetrack.pgm

In [11]:
metadata = {
    "image": "racetrack.pgm",
    "resolution": 0.01,
    "origin": [0.0, 0.0, 0.0],
    "occupied_thresh": 0.65,
    "free_thresh": 0.196,
    "negate": 0
}

In [12]:
metadata

{'image': 'racetrack.pgm',
 'resolution': 0.01,
 'origin': [0.0, 0.0, 0.0],
 'occupied_thresh': 0.65,
 'free_thresh': 0.196,
 'negate': 0}

In [13]:
import yaml
with open("racetrack.yaml", "w") as fh:
    yaml.dump(metadata, fh)